In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '' 

Using TensorFlow backend.


In [2]:
def target(value):
    if value>=0.5:
        return 1
    else:
        return 0

In [3]:
def preprocess(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [4]:
#Convert a list to a tuple
def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

In [5]:
def getData(cleaned_data, data_size):
    if(cleaned_data == "1"): # get cleaned and preprosessed data
        return pd.read_csv('train_preprocessed.csv',nrows=data_size) 
    else:
        return pd.read_csv('train.csv',nrows=data_size)

In [6]:
def print_parameters(dct):
    print("Parameters:")
    for item, amount in dct.items():
        print("{} ({})".format(item, amount))


In [7]:
def prepareData(train_df, max_words, max_len):  
    train_df =  train_df[['id','target','comment_text']]
    train_df['truth'] = train_df['target'].apply(target)
    train_df['comment_text'] = preprocess(train_df['comment_text'])
    
    X = train_df.comment_text
    Y = train_df.truth
    le = LabelEncoder()
    Y = le.fit_transform(Y)
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)
    
    tok = Tokenizer(num_words=max_words)
    tok.fit_on_texts(X_train)
    sequences = tok.texts_to_sequences(X_train)
    sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
    
    return (sequences_matrix, Y_train)

In [8]:
#  BATCH_SIZE, LSTM_UNITS, MAX_LEN, MAX_WORDS, DROPOUT, BIDIRECTIONAL, DOUBLE_LSTM_LAYER

def build_rnn(max_len , max_words, lstm_units, dropout, bidirectional, double_ltsm):
#     calculate Params
    DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

#     build RNN
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    
    if(double_ltsm):
        if(bidirectional):   
            layer = Bidirectional(LSTM(lstm_units, return_sequences=True))(layer)
        else:
            layer = LSTM(lstm_units, return_sequences=True)(layer)
        
    if(bidirectional):   
        layer = Bidirectional(LSTM(lstm_units))(layer)
    else:
        layer = LSTM(lstm_units)(layer)
        
    
    layer = Dense(DENSE_HIDDEN_UNITS, name='Dense')(layer)
    layer = Activation('relu')(layer)
    
    if(dropout):
        layer = Dropout(0.3)(layer)
    
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

###  grid search - every combination will be trained

In [9]:
# feature sellection variables
CLEANED_DATA = [True,
                False
               ]
DATA_ROWS = 200000

# Parameter tuning variablesmax_words = [2000] #
BATCH_SIZE = [512] # How many comments are used for each training cycle 
LSTM_UNITS = [
#     64, 
    128
    ] # how many units are in the LSTM

EPOCHS = [4] # num of times to go over entire training data set
MAX_LEN = [150] # numder of words allowed before the comment is cut off
MAX_WORDS = [2000] # number of features (if word isn't found, it's skipped)
DROPOUT = [
    True, 
#     False
] # dropout layer ratio 0 for so drop out

BIDIRECTIONAL = [
    True, 
    False
] # Whether or not to use the bidirectional STML layers
DOUBLE_LSTM_LAYER = [
    True, 
    False
] # Whether or not to use the bidirectional STML layers

help_display = dict(zip(
    ['CLEANED_DATA','DATA_ROWS','BATCH_SIZE', 'LSTM_UNITS','EPOCHS', 'MAX_LEN', 'MAX_WORDS','DROPOUT', 'BIDIRECTIONAL', 'DOUBLE_LSTM_LAYER'],
    [CLEANED_DATA,DATA_ROWS,BATCH_SIZE, LSTM_UNITS,EPOCHS, MAX_LEN, MAX_WORDS,DROPOUT, BIDIRECTIONAL, DOUBLE_LSTM_LAYER]
))


parameter_grid = np.array(np.meshgrid(CLEANED_DATA,
                                      DATA_ROWS,
                                      BATCH_SIZE, 
                                      LSTM_UNITS,
                                      EPOCHS, 
                                      MAX_LEN, 
                                      MAX_WORDS,
                                      DROPOUT, 
                                      BIDIRECTIONAL, 
                                      DOUBLE_LSTM_LAYER
                                     )).T.reshape(-1,10)

# print('bat, unit, epoc, len, words, drop, bid, 2-layers')
print_parameters(help_display)

Parameters:
CLEANED_DATA ([True, False])
DATA_ROWS (200000)
BATCH_SIZE ([512])
LSTM_UNITS ([128])
EPOCHS ([4])
MAX_LEN ([150])
MAX_WORDS ([2000])
DROPOUT ([True])
BIDIRECTIONAL ([True, False])
DOUBLE_LSTM_LAYER ([True, False])


In [10]:
for use_cleaned_data in CLEANED_DATA:
    train_df = getData(use_cleaned_data, DATA_ROWS)

    for row in parameter_grid:

        (CLEANED_DATA,DATA_ROWS,BATCH_SIZE, 
        LSTM_UNITS,EPOCHS, MAX_LEN, MAX_WORDS,
        DROPOUT, BIDIRECTIONAL, DOUBLE_LSTM_LAYER) = totuple(row)

        #Print display for tracking parameters
        help_display = dict(zip(
            ['CLEANED_DATA','DATA_ROWS','BATCH_SIZE', 'LSTM_UNITS','EPOCHS', 'MAX_LEN', 'MAX_WORDS','DROPOUT', 'BIDIRECTIONAL', 'DOUBLE_LSTM_LAYER'],
            [CLEANED_DATA,DATA_ROWS,BATCH_SIZE, LSTM_UNITS,EPOCHS, MAX_LEN, MAX_WORDS,DROPOUT, BIDIRECTIONAL, DOUBLE_LSTM_LAYER]
        ))
        print_parameters(help_display)
        sequences_matrix, Y_train = prepareData(train_df, MAX_WORDS, MAX_LEN) # get sequence matrix
    #     max_len , max_words, lstm_units, dropout, bidirectional, double_ltsm
        model = build_rnn(MAX_LEN, MAX_WORDS, LSTM_UNITS, DROPOUT, BIDIRECTIONAL, DOUBLE_LSTM_LAYER)

        print_parameters(help_display)
        print (model.summary())
        model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

        model.fit(sequences_matrix,Y_train,batch_size=128,epochs=5,
                  validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])
    
    

Parameters:
CLEANED_DATA (1)
DATA_ROWS (200000)
BATCH_SIZE (512)
LSTM_UNITS (128)
EPOCHS (4)
MAX_LEN (150)
MAX_WORDS (2000)
DROPOUT (1)
BIDIRECTIONAL (1)
DOUBLE_LSTM_LAYER (1)


E:\Users\avnee\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Users\avnee\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Parameters:
CLEANED_DATA (1)
DATA_ROWS (200000)
BATCH_SIZE (512)
LSTM_UNITS (128)
EPOCHS (4)
MAX_LEN (150)
MAX_WORDS (2000)
DROPOUT (1)
BIDIRECTIONAL (1)
DOUBLE_LSTM_LAYER (1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           100000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 256)          183296    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240    
___________________________________________

Train on 128000 samples, validate on 32000 samples
Epoch 1/5
128000/128000 [==============================] - 488s 4ms/step - loss: 0.2321 - acc: 0.9293 - val_loss: 0.2068 - val_acc: 0.9341
Epoch 2/5
128000/128000 [==============================] - 486s 4ms/step - loss: 0.2045 - acc: 0.9357 - val_loss: 0.1974 - val_acc: 0.9369
Epoch 3/5
128000/128000 [==============================] - 485s 4ms/step - loss: 0.1990 - acc: 0.9364 - val_loss: 0.2042 - val_acc: 0.9327
Parameters:
CLEANED_DATA (1)
DATA_ROWS (200000)
BATCH_SIZE (512)
LSTM_UNITS (128)
EPOCHS (4)
MAX_LEN (150)
MAX_WORDS (2000)
DROPOUT (1)
BIDIRECTIONAL (1)
DOUBLE_LSTM_LAYER (0)
Parameters:
CLEANED_DATA (1)
DATA_ROWS (200000)
BATCH_SIZE (512)
LSTM_UNITS (128)
EPOCHS (4)
MAX_LEN (150)
MAX_WORDS (2000)
DROPOUT (1)
BIDIRECTIONAL (1)
DOUBLE_LSTM_LAYER (0)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)        

Train on 128000 samples, validate on 32000 samples
Epoch 1/5
128000/128000 [==============================] - 247s 2ms/step - loss: 0.2390 - acc: 0.9278 - val_loss: 0.2112 - val_acc: 0.9349
Epoch 2/5
128000/128000 [==============================] - 245s 2ms/step - loss: 0.2085 - acc: 0.9338 - val_loss: 0.1968 - val_acc: 0.9375
Epoch 3/5
128000/128000 [==============================] - 245s 2ms/step - loss: 0.2010 - acc: 0.9352 - val_loss: 0.1999 - val_acc: 0.9382
Parameters:
CLEANED_DATA (1)
DATA_ROWS (200000)
BATCH_SIZE (512)
LSTM_UNITS (128)
EPOCHS (4)
MAX_LEN (150)
MAX_WORDS (2000)
DROPOUT (1)
BIDIRECTIONAL (1)
DOUBLE_LSTM_LAYER (1)
Parameters:
CLEANED_DATA (1)
DATA_ROWS (200000)
BATCH_SIZE (512)
LSTM_UNITS (128)
EPOCHS (4)
MAX_LEN (150)
MAX_WORDS (2000)
DROPOUT (1)
BIDIRECTIONAL (1)
DOUBLE_LSTM_LAYER (1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)        

Train on 128000 samples, validate on 32000 samples
Epoch 1/5
128000/128000 [==============================] - 491s 4ms/step - loss: 0.2339 - acc: 0.9298 - val_loss: 0.2120 - val_acc: 0.9335
Epoch 2/5
128000/128000 [==============================] - 489s 4ms/step - loss: 0.2083 - acc: 0.9341 - val_loss: 0.2072 - val_acc: 0.9338
Epoch 3/5
128000/128000 [==============================] - 488s 4ms/step - loss: 0.2010 - acc: 0.9353 - val_loss: 0.2011 - val_acc: 0.9353
Epoch 4/5
128000/128000 [==============================] - 487s 4ms/step - loss: 0.1956 - acc: 0.9368 - val_loss: 0.2011 - val_acc: 0.9351
Parameters:
CLEANED_DATA (1)
DATA_ROWS (200000)
BATCH_SIZE (512)
LSTM_UNITS (128)
EPOCHS (4)
MAX_LEN (150)
MAX_WORDS (2000)
DROPOUT (1)
BIDIRECTIONAL (1)
DOUBLE_LSTM_LAYER (0)
Parameters:
CLEANED_DATA (1)
DATA_ROWS (200000)
BATCH_SIZE (512)
LSTM_UNITS (128)
EPOCHS (4)
MAX_LEN (150)
MAX_WORDS (2000)
DROPOUT (1)
BIDIRECTIONAL (1)
DOUBLE_LSTM_LAYER (0)
_________________________________________

Train on 128000 samples, validate on 32000 samples
Epoch 1/5
128000/128000 [==============================] - 252s 2ms/step - loss: 0.2504 - acc: 0.9270 - val_loss: 0.2166 - val_acc: 0.9310
Epoch 2/5
128000/128000 [==============================] - 248s 2ms/step - loss: 0.2126 - acc: 0.9336 - val_loss: 0.2210 - val_acc: 0.9354


In [ ]:
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
    model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
              validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']